In [ ]:
library(edgeR)
library(Seurat)
library(ggvenn)
library(pheatmap)
library(dplyr)

# source("~/aging_functions.R")
# library(UpSetR)
# library(tidyverse)
# library(SummarizedExperiment)
# library(limma)
# library(ggplotify)
# library(ComplexHeatmap)

### Loading data and pseudobulking

In [ ]:
SCC <- c(
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_B18_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_E15_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_F21_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_P13_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_P30_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_B18_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_P13_SCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_P30_SCC_lr_sig_scores.csv'
    )

BCC <- c(
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_B18_BCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_E15_BCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_F21_BCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_bcc_B18_BCC_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_bcc_D12_BCC_lr_sig_scores.csv'
    )

mela <- c(
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_6767_Mel_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_21031_Mel_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_48974_Mel_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_66487_Mel_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_6747-085P_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_66487-1A_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_48974-2B_lr_sig_scores.csv',
    '/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_21031-08TB_lr_sig_scores.csv'
    )

In [ ]:
# read in each csv
scc <- lapply(SCC, read.csv, row.names=1)
bcc <- lapply(BCC, read.csv, row.names=1)
mel <- lapply(mela, read.csv, row.names=1)

In [ ]:
for (i in 1:length(scc)){
    scc[[i]] <- CreateSeuratObject(counts = t(scc[[i]]))
    # randomly assign a number 1-3 to each cell
    set.seed(123)
    scc[[i]]$group <- sample(1:3, nrow(scc[[i]]), replace = TRUE)
}

for (i in 1:length(bcc)){
    bcc[[i]] <- CreateSeuratObject(counts = t(bcc[[i]]))
    # randomly assign a number 1-3 to each cell
    set.seed(123)
    bcc[[i]]$group <- sample(1:3, nrow(bcc[[i]]), replace = TRUE)
}

for (i in 1:length(mel)){
    mel[[i]] <- CreateSeuratObject(counts = t(mel[[i]]))
    # randomly assign a number 1-3 to each cell
    set.seed(123)
    mel[[i]]$group <- sample(1:3, nrow(mel[[i]]), replace = TRUE)
}

In [ ]:
scc_vis <- merge(scc[[1]], y = scc[2:5])
scc_vis <- JoinLayers(scc_vis)
scc_vis$cancer <- 'SCC'
scc_cos <- merge(scc[[6]], y = scc[7:8])
scc_cos <- JoinLayers(scc_cos)
scc_cos$cancer <- 'SCC'
bcc_vis <- merge(bcc[[1]], y = bcc[2:3])
bcc_vis <- JoinLayers(bcc_vis)
bcc_vis$cancer <- 'BCC'
bcc_cos <- merge(bcc[[4]], y = bcc[5])
bcc_cos <- JoinLayers(bcc_cos)
bcc_cos$cancer <- 'BCC'
mel_vis <- merge(mel[[1]], y = mel[2:4])
mel_vis <- JoinLayers(mel_vis)
mel_vis$cancer <- 'Mel'
mel_cos <- merge(mel[[5]], y = mel[6:8])
mel_cos <- JoinLayers(mel_cos)
mel_cos$cancer <- 'Mel'

vis <- merge(scc_vis, y = c(bcc_vis, mel_vis))
vis <- JoinLayers(vis)
cos <- merge(scc_cos, y = c(bcc_cos, mel_cos))
cos <- JoinLayers(cos)

### Combine SCC and BCC

In [ ]:
vis$cancer_SCC_BCC <- ifelse(vis$cancer %in% c("SCC", "BCC"), "Others", vis$cancer)
cos$cancer_SCC_BCC <- ifelse(cos$cancer %in% c("SCC", "BCC"), "Others", cos$cancer)

In [ ]:
vis_pb <- Seurat2PB(vis, sample='cancer_SCC_BCC', cluster='group')
cos_pb <- Seurat2PB(cos, sample='cancer_SCC_BCC', cluster='group')



keep.genes <- filterByExpr(vis_pb, group=vis_pb$samples$cluster, min.count=1, min.total.count=3)
print("For Visium")
table(keep.genes)
vis_pb <- vis_pb[keep.genes, , keep=FALSE]

keep.genes <- filterByExpr(cos_pb, group=cos_pb$samples$cluster, min.count=10, min.total.count=20)
print("For CosMx")
table(keep.genes)
cos_pb <- cos_pb[keep.genes, , keep=FALSE]



vis_pb <- normLibSizes(vis_pb)
cos_pb <- normLibSizes(cos_pb)

In [ ]:
cluster <- as.factor(vis_pb$samples$sample)
plotMDS(vis_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

cluster <- as.factor(cos_pb$samples$sample)
plotMDS(cos_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

In [ ]:
# Set up the design matrix
cancer <- factor(vis_pb$samples$sample)
group <- factor(vis_pb$samples$cluster)
vis_design <-  model.matrix(~cancer)
colnames(vis_design) <- levels(cancer)
colnames(vis_design)[1] <- "Int"

cancer <- factor(cos_pb$samples$sample)
group <- factor(cos_pb$samples$cluster)
cos_design <-  model.matrix(~cancer)
colnames(cos_design) <- levels(cancer)
colnames(cos_design)[1] <- "Int"



# Estimate dispersion
vis_pb <- estimateDisp(vis_pb, vis_design, robust=TRUE)
vis_pb$common.dispersion
plotBCV(vis_pb)

cos_pb <- estimateDisp(cos_pb, cos_design, robust=TRUE)
cos_pb$common.dispersion
plotBCV(cos_pb)


# Fit the glm model
vis_fit <- glmQLFit(vis_pb, vis_design, robust=TRUE)
plotQLDisp(vis_fit)

cos_fit <- glmQLFit(cos_pb, cos_design, robust=TRUE)
plotQLDisp(cos_fit)



# Plot the design matrix
ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(vis_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(vis_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(cos_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(cos_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr



# Visualise the top DE genes
vis_qlf <- list()
for(i in 1:ncls) {
    vis_qlf[[i]] <- glmQLFTest(vis_fit, contrast=contr[,i])
    vis_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(vis_qlf[[1]])

cos_qlf <- list()
for(i in 1:ncls) {
    cos_qlf[[i]] <- glmQLFTest(cos_fit, contrast=contr[,i])
    cos_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(cos_qlf[[1]])

In [ ]:
vis_table <- vis_qlf[[2]]$table
vis_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
vis_table$regulate[vis_table$logFC > 0 & vis_table$PValue < 0.05] <- 'Up'
vis_table$regulate[vis_table$logFC < 0 & vis_table$PValue < 0.05] <- 'Down'

cos_table <- cos_qlf[[2]]$table
cos_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
cos_table$regulate[cos_table$logFC > 0 & cos_table$PValue < 0.05] <- 'Up'
cos_table$regulate[cos_table$logFC < 0 & cos_table$PValue < 0.05] <- 'Down'


cos_up <- row.names(cos_table[cos_table$regulate == 'Up',])
vis_up <- row.names(vis_table[vis_table$regulate == 'Up',])
cos_down <- row.names(cos_table[cos_table$regulate == 'Down',])
vis_down <- row.names(vis_table[vis_table$regulate == 'Down',])

##### <font color= "blue"> The directionality is randomly assigned by edgeR; and here the directionality was with respect to the group "Other", so it is reveresed when plotting w.r.t to "Melanoma"

In [ ]:
pdf("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/Individual_panels_fig9_10/DELRs_Venn.pdf",height=6.5, width=6.5)
ggvenn(
    list('Cos Down' = cos_up, 'Vis Down' = vis_up, 'Cos Up' = cos_down, 'Vis Up' = vis_down), text_size = 5.5
    )
dev.off()

In [ ]:

save_pheatmap_pdf <- function(x, filename, width=25, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}


In [ ]:
top <- 20
topMarkers <- list()
for(i in 1:ncls) {
    ord <- order(vis_qlf[[i]]$table$PValue, decreasing=FALSE)
    up <- vis_qlf[[i]]$table$logFC > 0
    topMarkers[[i]] <- rownames(vis_pb)[ord[up][1:top]]
}
topMarkers <- unique(unlist(topMarkers))

lcpm <- cpm(vis_pb, log=TRUE)
annot <- data.frame(cluster=cluster)
rownames(annot) <- colnames(vis_pb)
ann_colors <- list(cluster=1:2)
names(ann_colors$cluster) <- paste0(levels(cluster))

plot_pheatmap <- pheatmap::pheatmap(lcpm[topMarkers, ], breaks=seq(-2,2,length.out=101), 
                                    color=colorRampPalette(c("blue","white","red"))(100), scale="row",
                                    cluster_cols=TRUE, border_color="NA", fontsize_row=27.5,
                                    treeheight_row=70, treeheight_col=70, fontsize = 15,
                                    clustering_method="ward.D2", show_colnames=FALSE,
                                    annotation_col=annot, annotation_colors=ann_colors)
save_pheatmap_pdf(plot_pheatmap, "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/Individual_panels_fig9_10/Vis_DELRs_heatmap.pdf", width=12.5, height=17.5)

vis_lcpm <- lcpm


In [ ]:
top <- 20
topMarkers <- list()
for(i in 1:ncls) {
    ord <- order(cos_qlf[[i]]$table$PValue, decreasing=FALSE)
    up <- cos_qlf[[i]]$table$logFC > 0
    topMarkers[[i]] <- rownames(cos_pb)[ord[up][1:top]]
}
topMarkers <- unique(unlist(topMarkers))

lcpm <- cpm(cos_pb, log=TRUE)
annot <- data.frame(cluster=cluster)
rownames(annot) <- colnames(cos_pb)
ann_colors <- list(cluster=1:2)
names(ann_colors$cluster) <- paste0(levels(cluster))
plot_pheatmap <- pheatmap::pheatmap(lcpm[topMarkers, ], breaks=seq(-2,2,length.out=101), 
                                    color=colorRampPalette(c("blue","white","red"))(100), scale="row",
                                    cluster_cols=TRUE, border_color="NA", fontsize_row=27.5,
                                    treeheight_row=70, treeheight_col=70,
                                    clustering_method="ward.D2", show_colnames=FALSE,
                                    annotation_col=annot, annotation_colors=ann_colors)
save_pheatmap_pdf(plot_pheatmap, "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/Individual_panels_fig9_10/CosMx_DELRs_heatmap.pdf", width=12.5, height=17.5)

cos_lcpm <- lcpm

In [ ]:
print(intersect(cos_up, vis_up))

In [ ]:
print(intersect(cos_down, vis_down))

In [ ]:
saveRDS(intersect(cos_down, vis_down),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsMelanoma_upreg_2way_comparison.rds")
saveRDS(intersect(cos_up, vis_up),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsMelanoma_downreg_2way_comparison.rds")

In [ ]:
mel_up <- readRDS("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsMelanoma_upreg_2way_comparison.rds")
mel_up
mel_down <- readRDS("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsMelanoma_downreg_2way_comparison.rds")
mel_down

### Combine Melanoma and BCC

In [ ]:
vis$cancer_Mel_BCC <- ifelse(vis$cancer %in% c("Mel", "BCC"), "Others", vis$cancer)
cos$cancer_Mel_BCC <- ifelse(cos$cancer %in% c("Mel", "BCC"), "Others", cos$cancer)

In [ ]:
vis_pb <- Seurat2PB(vis, sample='cancer_Mel_BCC', cluster='group')
cos_pb <- Seurat2PB(cos, sample='cancer_Mel_BCC', cluster='group')



keep.genes <- filterByExpr(vis_pb, group=vis_pb$samples$cluster, min.count=1, min.total.count=3)
print("For Visium")
table(keep.genes)
vis_pb <- vis_pb[keep.genes, , keep=FALSE]

keep.genes <- filterByExpr(cos_pb, group=cos_pb$samples$cluster, min.count=10, min.total.count=20)
print("For CosMx")
table(keep.genes)
cos_pb <- cos_pb[keep.genes, , keep=FALSE]



vis_pb <- normLibSizes(vis_pb)
cos_pb <- normLibSizes(cos_pb)

In [ ]:
cluster <- as.factor(vis_pb$samples$sample)
plotMDS(vis_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

cluster <- as.factor(cos_pb$samples$sample)
plotMDS(cos_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

In [ ]:
# Set up the design matrix
cancer <- factor(vis_pb$samples$sample)
group <- factor(vis_pb$samples$cluster)
vis_design <-  model.matrix(~cancer)
colnames(vis_design) <- levels(cancer)
colnames(vis_design)[1] <- "Int"

cancer <- factor(cos_pb$samples$sample)
group <- factor(cos_pb$samples$cluster)
cos_design <-  model.matrix(~cancer)
colnames(cos_design) <- levels(cancer)
colnames(cos_design)[1] <- "Int"



# Estimate dispersion
vis_pb <- estimateDisp(vis_pb, vis_design, robust=TRUE)
vis_pb$common.dispersion
plotBCV(vis_pb)

cos_pb <- estimateDisp(cos_pb, cos_design, robust=TRUE)
cos_pb$common.dispersion
plotBCV(cos_pb)


# Fit the glm model
vis_fit <- glmQLFit(vis_pb, vis_design, robust=TRUE)
plotQLDisp(vis_fit)

cos_fit <- glmQLFit(cos_pb, cos_design, robust=TRUE)
plotQLDisp(cos_fit)



# Plot the design matrix
ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(vis_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(vis_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(cos_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(cos_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr



# Visualise the top DE genes
vis_qlf <- list()
for(i in 1:ncls) {
    vis_qlf[[i]] <- glmQLFTest(vis_fit, contrast=contr[,i])
    vis_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(vis_qlf[[1]])

cos_qlf <- list()
for(i in 1:ncls) {
    cos_qlf[[i]] <- glmQLFTest(cos_fit, contrast=contr[,i])
    cos_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(cos_qlf[[1]])

In [ ]:
vis_table <- vis_qlf[[2]]$table
vis_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
vis_table$regulate[vis_table$logFC > 0 & vis_table$PValue < 0.05] <- 'Up'
vis_table$regulate[vis_table$logFC < 0 & vis_table$PValue < 0.05] <- 'Down'

cos_table <- cos_qlf[[2]]$table
cos_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
cos_table$regulate[cos_table$logFC > 0 & cos_table$PValue < 0.05] <- 'Up'
cos_table$regulate[cos_table$logFC < 0 & cos_table$PValue < 0.05] <- 'Down'


cos_up <- row.names(cos_table[cos_table$regulate == 'Up',])
vis_up <- row.names(vis_table[vis_table$regulate == 'Up',])
cos_down <- row.names(cos_table[cos_table$regulate == 'Down',])
vis_down <- row.names(vis_table[vis_table$regulate == 'Down',])

In [ ]:
print(intersect(cos_up,vis_up))

In [ ]:
print(intersect(cos_down,vis_down))

In [ ]:
saveRDS(intersect(cos_up, vis_up),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsSCC_upreg_2way_comparison.rds")
saveRDS(intersect(cos_down, vis_down),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsSCC_downreg_2way_comparison.rds")

### Combine Melanoma and SCC

In [ ]:
vis$cancer_Mel_SCC <- ifelse(vis$cancer %in% c("Mel", "SCC"), "Others", vis$cancer)
cos$cancer_Mel_SCC <- ifelse(cos$cancer %in% c("Mel", "SCC"), "Others", cos$cancer)

In [ ]:
vis_pb <- Seurat2PB(vis, sample='cancer_Mel_SCC', cluster='group')
cos_pb <- Seurat2PB(cos, sample='cancer_Mel_SCC', cluster='group')



keep.genes <- filterByExpr(vis_pb, group=vis_pb$samples$cluster, min.count=1, min.total.count=3)
print("For Visium")
table(keep.genes)
vis_pb <- vis_pb[keep.genes, , keep=FALSE]

keep.genes <- filterByExpr(cos_pb, group=cos_pb$samples$cluster, min.count=10, min.total.count=20)
print("For CosMx")
table(keep.genes)
cos_pb <- cos_pb[keep.genes, , keep=FALSE]



vis_pb <- normLibSizes(vis_pb)
cos_pb <- normLibSizes(cos_pb)

In [ ]:
cluster <- as.factor(vis_pb$samples$sample)
plotMDS(vis_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

cluster <- as.factor(cos_pb$samples$sample)
plotMDS(cos_pb, pch=16, col=c(2:8)[cluster], main="MDS")
legend("topleft", legend=paste0("cluster",levels(cluster)), pch=16, col=2:8, cex=0.8)

In [ ]:
# Set up the design matrix
cancer <- factor(vis_pb$samples$sample)
group <- factor(vis_pb$samples$cluster)
vis_design <-  model.matrix(~cancer)
colnames(vis_design) <- levels(cancer)
colnames(vis_design)[1] <- "Int"

cancer <- factor(cos_pb$samples$sample)
group <- factor(cos_pb$samples$cluster)
cos_design <-  model.matrix(~cancer)
colnames(cos_design) <- levels(cancer)
colnames(cos_design)[1] <- "Int"



# Estimate dispersion
vis_pb <- estimateDisp(vis_pb, vis_design, robust=TRUE)
vis_pb$common.dispersion
plotBCV(vis_pb)

cos_pb <- estimateDisp(cos_pb, cos_design, robust=TRUE)
cos_pb$common.dispersion
plotBCV(cos_pb)


# Fit the glm model
vis_fit <- glmQLFit(vis_pb, vis_design, robust=TRUE)
plotQLDisp(vis_fit)

cos_fit <- glmQLFit(cos_pb, cos_design, robust=TRUE)
plotQLDisp(cos_fit)



# Plot the design matrix
ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(vis_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(vis_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(cos_design)-ncls, ncls))
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(cos_design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr



# Visualise the top DE genes
vis_qlf <- list()
for(i in 1:ncls) {
    vis_qlf[[i]] <- glmQLFTest(vis_fit, contrast=contr[,i])
    vis_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(vis_qlf[[1]])

cos_qlf <- list()
for(i in 1:ncls) {
    cos_qlf[[i]] <- glmQLFTest(cos_fit, contrast=contr[,i])
    cos_qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
topTags(cos_qlf[[1]])

In [ ]:
vis_table <- vis_qlf[[2]]$table
vis_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
vis_table$regulate[vis_table$logFC > 0 & vis_table$PValue < 0.05] <- 'Up'
vis_table$regulate[vis_table$logFC < 0 & vis_table$PValue < 0.05] <- 'Down'

cos_table <- cos_qlf[[2]]$table
cos_table$regulate <- 'Normal'
# set regulate to up if logfc > 1.2 and pvalue < 0.05
cos_table$regulate[cos_table$logFC > 0 & cos_table$PValue < 0.05] <- 'Up'
cos_table$regulate[cos_table$logFC < 0 & cos_table$PValue < 0.05] <- 'Down'


cos_up <- row.names(cos_table[cos_table$regulate == 'Up',])
vis_up <- row.names(vis_table[vis_table$regulate == 'Up',])
cos_down <- row.names(cos_table[cos_table$regulate == 'Down',])
vis_down <- row.names(vis_table[vis_table$regulate == 'Down',])

In [ ]:
print(intersect(cos_up,vis_up))

In [ ]:
print(intersect(cos_down,vis_down))

In [ ]:
saveRDS(intersect(cos_down, vis_down),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsBCC_upreg_2way_comparison.rds")
saveRDS(intersect(cos_up, vis_up),"/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsBCC_downreg_2way_comparison.rds")

### Unique interactions

In [ ]:
scc_up <- readRDS("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsSCC_upreg_2way_comparison.rds")
bcc_up <- readRDS("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsBCC_upreg_2way_comparison.rds")
mel_up <- readRDS("/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/edgeR_LRsMelanoma_upreg_2way_comparison.rds")

In [ ]:
print("SCC===============")
print(scc_up)
print("BCC===============")
print(bcc_up)
print("Mel===============")
print(mel_up)

In [ ]:
scc_up <- setdiff(scc_up,c(bcc_up,mel_up))
bcc_up <- setdiff(bcc_up,c(scc_up,mel_up))
mel_up <- setdiff(mel_up,c(bcc_up,scc_up))

In [ ]:
print("SCC===============")
print(scc_up)
print("BCC===============")
print(bcc_up)
print("Mel===============")
print(mel_up)

### LR Heatmap

In [ ]:
vis_B18_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_B18_SCC_lr_sig_scores.csv', row.names = 1)
vis_E15_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_E15_SCC_lr_sig_scores.csv', row.names = 1)
vis_F21_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_F21_SCC_lr_sig_scores.csv', row.names = 1)
vis_P13_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_P13_SCC_lr_sig_scores.csv', row.names = 1)
vis_P30_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_scc_P30_SCC_lr_sig_scores.csv', row.names = 1)
cos_B18_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_B18_SCC_lr_sig_scores.csv', row.names = 1)
cos_P13_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_P13_SCC_lr_sig_scores.csv', row.names = 1)
cos_P30_SCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_scc_P30_SCC_lr_sig_scores.csv', row.names = 1)

vis_B18_BCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_B18_BCC_lr_sig_scores.csv', row.names = 1)
vis_E15_BCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_E15_BCC_lr_sig_scores.csv', row.names = 1)
vis_F21_BCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_bcc_F21_BCC_lr_sig_scores.csv', row.names = 1)
cos_B18_BCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_bcc_B18_BCC_lr_sig_scores.csv', row.names = 1)
cos_D12_BCC <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_bcc_D12_BCC_lr_sig_scores.csv', row.names = 1)

vis_6767_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_6767_Mel_lr_sig_scores.csv', row.names = 1)
vis_21031_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_21031_Mel_lr_sig_scores.csv', row.names = 1)
vis_48974_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_48974_Mel_lr_sig_scores.csv', row.names = 1)
vis_66487_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/visium_mela_66487_Mel_lr_sig_scores.csv', row.names = 1)
cos_6747_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_6747-085P_lr_sig_scores.csv', row.names = 1)
cos_66487_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_66487-1A_lr_sig_scores.csv', row.names = 1)
cos_48974_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_48974-2B_lr_sig_scores.csv', row.names = 1)
cos_21031_Mel <- read.csv('/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/cosmx_mela_21031-08TB_lr_sig_scores.csv', row.names = 1)

In [ ]:
# Calculate the average of each column for each data frame
avg_vis_B18_SCC <- as.data.frame(colMeans(vis_B18_SCC, na.rm = TRUE))
colnames(avg_vis_B18_SCC) <- "B18_SCC"
avg_vis_E15_SCC <- as.data.frame(colMeans(vis_E15_SCC, na.rm = TRUE))
colnames(avg_vis_E15_SCC) <- "E15_SCC"
avg_vis_F21_SCC <- as.data.frame(colMeans(vis_F21_SCC, na.rm = TRUE))
colnames(avg_vis_F21_SCC) <- "F21_SCC"
avg_vis_P13_SCC <- as.data.frame(colMeans(vis_P13_SCC, na.rm = TRUE))
colnames(avg_vis_P13_SCC) <- "P13_SCC"
avg_vis_P30_SCC <- as.data.frame(colMeans(vis_P30_SCC, na.rm = TRUE))
colnames(avg_vis_P30_SCC) <- "P30_SCC"
avg_cos_B18_SCC <- as.data.frame(colMeans(cos_B18_SCC, na.rm = TRUE))
colnames(avg_cos_B18_SCC) <- "B18_SCC"
avg_cos_P13_SCC <- as.data.frame(colMeans(cos_P13_SCC, na.rm = TRUE))
colnames(avg_cos_P13_SCC) <- "P13_SCC"
avg_cos_P30_SCC <- as.data.frame(colMeans(cos_P30_SCC, na.rm = TRUE))
colnames(avg_cos_P30_SCC) <- "P30_SCC"

avg_vis_B18_BCC <- as.data.frame(colMeans(vis_B18_BCC, na.rm = TRUE))
colnames(avg_vis_B18_BCC) <- "B18_BCC"
avg_vis_E15_BCC <- as.data.frame(colMeans(vis_E15_BCC, na.rm = TRUE))
colnames(avg_vis_E15_BCC) <- "E15_BCC"
avg_vis_F21_BCC <- as.data.frame(colMeans(vis_F21_BCC, na.rm = TRUE))
colnames(avg_vis_F21_BCC) <- "F21_BCC"
avg_cos_B18_BCC <- as.data.frame(colMeans(cos_B18_BCC, na.rm = TRUE))
colnames(avg_cos_B18_BCC) <- "B18_BCC"
avg_cos_D12_BCC <- as.data.frame(colMeans(cos_D12_BCC, na.rm = TRUE))
colnames(avg_cos_D12_BCC) <- "D12_BCC"

avg_vis_6767_Mel <- as.data.frame(colMeans(vis_6767_Mel, na.rm = TRUE))
colnames(avg_vis_6767_Mel) <- "6767_Mel"
avg_vis_21031_Mel <- as.data.frame(colMeans(vis_21031_Mel, na.rm = TRUE))
colnames(avg_vis_21031_Mel) <- "21031_Mel"
avg_vis_48974_Mel <- as.data.frame(colMeans(vis_48974_Mel, na.rm = TRUE))
colnames(avg_vis_48974_Mel) <- "48974_Mel"
avg_vis_66487_Mel <- as.data.frame(colMeans(vis_66487_Mel, na.rm = TRUE))
colnames(avg_vis_66487_Mel) <- "66487_Mel"
avg_cos_6747_Mel <- as.data.frame(colMeans(cos_6747_Mel, na.rm = TRUE))
colnames(avg_cos_6747_Mel) <- "6747_Mel"
avg_cos_66487_Mel <- as.data.frame(colMeans(cos_66487_Mel, na.rm = TRUE))
colnames(avg_cos_66487_Mel) <- "66487_Mel"
avg_cos_48974_Mel <- as.data.frame(colMeans(cos_48974_Mel, na.rm = TRUE))
colnames(avg_cos_48974_Mel) <- "48974_Mel"
avg_cos_21031_Mel <- as.data.frame(colMeans(cos_21031_Mel, na.rm = TRUE))
colnames(avg_cos_21031_Mel) <- "21031_Mel"

In [ ]:
multimerge <- function (mylist) {
    ## mimics a recursive merge or full outer join

    unames <- unique(unlist(lapply(mylist, rownames)))

    n <- length(unames)

    out <- lapply(mylist, function(df) {

    tmp <- matrix(nr = n, nc = ncol(df), dimnames = list(unames,colnames(df)))
    tmp[rownames(df), ] <- as.matrix(df)
    rm(df); gc()

    return(tmp)
    })

    stopifnot( all( sapply(out, function(x) identical(rownames(x), unames)) ) )

    bigout <- do.call(cbind, out)
    colnames(bigout) <- paste(rep(names(mylist), sapply(mylist, ncol)), unlist(sapply(mylist, colnames)), sep = "_")
    return(bigout)
}

# Combine all the data frames into one
l <- list(avg_vis_B18_SCC, avg_vis_E15_SCC, avg_vis_F21_SCC, avg_vis_P13_SCC, avg_vis_P30_SCC, avg_cos_B18_SCC, avg_cos_P13_SCC, avg_cos_P30_SCC, avg_vis_B18_BCC, avg_vis_E15_BCC, avg_vis_F21_BCC, avg_cos_B18_BCC, avg_cos_D12_BCC, avg_vis_6767_Mel, avg_vis_21031_Mel, avg_vis_48974_Mel, avg_vis_66487_Mel, avg_cos_6747_Mel, avg_cos_66487_Mel, avg_cos_48974_Mel, avg_cos_21031_Mel)

all_df <- multimerge(l)
colnames(all_df) <- c("vis_B18_SCC", "vis_E15_SCC", "vis_F21_SCC", "vis_P13_SCC", "vis_P30_SCC", "cos_B18_SCC", "cos_P13_SCC", "cos_P30_SCC", "vis_B18_BCC", "vis_E15_BCC", "vis_F21_BCC", "cos_B18_BCC", "cos_D12_BCC", "vis_6767_Mel", "vis_21031_Mel", "vis_48974_Mel", "vis_66487_Mel", "cos_6747_Mel", "cos_66487_Mel", "cos_48974_Mel", "cos_21031_Mel")

# make NA values 0
all_df[is.na(all_df)] <- 0

In [ ]:
library(stringr)

all_df_t <- as.data.frame(t(all_df))
all_df_t$Tech <- c(rep("Vis",5),rep("Cos",3),rep("Vis",3),rep("Cos",2),rep("Vis",4),rep("Cos",4))
all_df_t$Cancer <- c(rep("SCC",8),rep("BCC",5),rep("Mel",8))
all_df_t$sample <- str_extract(rownames(all_df_t), "(?<=_)[^_]+(?=_)")

In [ ]:
# Normalization and scaling function
normalize_and_scale <- function(data) {
  numeric_cols <- sapply(data, is.numeric)
  data[, numeric_cols] <- scale(data[, numeric_cols]) # Standardization
  data[, numeric_cols] <- apply(data[, numeric_cols], 2, function(x) (x - min(x)) / (max(x) - min(x))) # Normalization
  
  return(data)
}

# Group by the last two categorical columns and normalize/scale within groups
df_normalized <- all_df_t %>%
  group_by(Tech) %>%
  group_modify(~ normalize_and_scale(.x)) %>%
  ungroup()

df_normalized[is.na(df_normalized)] <- 0
df_normalized <- as.data.frame(df_normalized)
rownames(df_normalized) <- paste0(df_normalized$Tech,"_",df_normalized$sample,"_",df_normalized$Cancer)

df_normalized <- df_normalized %>%
                 mutate(Cancer = factor(Cancer, levels = c("BCC", "SCC", "Mel"))) %>%
                 arrange(Cancer) 

In [ ]:
options(repr.plot.height=7.5,repr.plot.width=15)

colors <- colorRampPalette(c("blue", "white", "gold"))(100)
annotation_colors <- list(
  Cancer = c("BCC" = "#FAA0A0", "SCC" = "#FF0000", "Mel" = "#800000"),
  Tech = c("Vis" = "Grey", "Cos" = "Black")
)
plot_pheatmap <- pheatmap::pheatmap(df_normalized[,gsub("-","_",c(bcc_up,scc_up,mel_up))], 
                   cluster_cols = F, 
                   cluster_rows = F,
                   border_color = "black",
                   gaps_row = c(5,13),
                   gaps_col = c(16,33),
                   annotation_row=df_normalized[c("Tech","Cancer")],
                   annotation_colors = annotation_colors,
                   color=colors,
                   fontsize_col=20,                 
                   angle_col = 90,
                   fontsize_row = 17.5)
save_pheatmap_pdf(plot_pheatmap, "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/Individual_panels_fig9_10/Cancer_specific_DELRs_heatmap.pdf", width=25, height=12.5)
